In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import hate_column_converter
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-10-23 22:44:31.150630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-23 22:44:31.150668: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [3]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
    #zip_ref.extractall("../../../../embeddings/")

In [2]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
y = new_dataframe['hate_speech']

/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-my-paper/lstm/hate_speech/../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numb

In [3]:
x = new_dataframe['instagram_comments']

In [4]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [5]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 11530/11530 [00:00<00:00, 392013.27it/s]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [7]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

2023-10-23 22:46:26.605664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-23 22:46:26.605703: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-23 22:46:26.605736: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-fbf4a0): /proc/driver/nvidia/version does not exist
2023-10-23 22:46:26.640565: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model = lstm_model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/15


2023-10-23 22:46:38.951869: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 22:46:38.970425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 22:46:39.060557: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 22:46:39.215343: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


 1/44 [..............................] - ETA: 2:28 - loss: 0.6150 - accuracy: 0.7734 - binary_accuracy: 0.7734 - f1_score: 0.2162

2023-10-23 22:46:40.275976: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


44/44 [==============================] - 26s 530ms/step - loss: 0.3946 - accuracy: 0.8870 - binary_accuracy: 0.8870 - f1_score: 0.0103 - val_loss: 0.3465 - val_accuracy: 0.8890 - val_binary_accuracy: 0.8890 - val_f1_score: 0.0000e+00
Epoch 2/15
44/44 [==============================] - 22s 504ms/step - loss: 0.3044 - accuracy: 0.8984 - binary_accuracy: 0.8984 - f1_score: 0.0105 - val_loss: 0.3054 - val_accuracy: 0.8904 - val_binary_accuracy: 0.8904 - val_f1_score: 0.0236
Epoch 3/15
44/44 [==============================] - 22s 498ms/step - loss: 0.2707 - accuracy: 0.8995 - binary_accuracy: 0.8995 - f1_score: 0.0708 - val_loss: 0.2794 - val_accuracy: 0.8954 - val_binary_accuracy: 0.8954 - val_f1_score: 0.1661
Epoch 4/15
44/44 [==============================] - 22s 502ms/step - loss: 0.2536 - accuracy: 0.9064 - binary_accuracy: 0.9064 - f1_score: 0.2122 - val_loss: 0.2698 - val_accuracy: 0.8975 - val_binary_accuracy: 0.8975 - val_f1_score: 0.3116
Epoch 5/15
44/44 [=========================

In [9]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

 2/11 [====>.........................] - ETA: 1s - loss: 0.2417 - accuracy: 0.9258 - binary_accuracy: 0.9258 - f1_score: 0.5000

11/11 [==============================] - 2s 161ms/step - loss: 0.2729 - accuracy: 0.9160 - binary_accuracy: 0.9160 - f1_score: 0.5020
loss :  0.27293476462364197
accuracy :  0.9160142540931702
binary_accuracy :  0.9160142540931702
f1_score :  0.5019952654838562



In [12]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)

: 